In [ ]:
# save a default checkpoint
import torch

from starry.utils.config import Configuration
from starry.utils.model_factory import loadModel


config = Configuration.createOrLoad('configs/testencoder.yaml')
model = loadModel(config['model'])

torch.save({'model': model.state_dict()}, config.localPath('test.chkpt'))


In [14]:
# deduce test
import torch

from starry.utils.config import Configuration
from starry.utils.model_factory import loadModel


config = Configuration.createOrLoad('training/melody/20220526-testencoder')
model = loadModel(config['model'])

checkpoint = torch.load(config.localPath(config['best']), map_location='cpu')
model.load_state_dict(checkpoint['model'])

model.eval()

input = torch.tensor([1, 2, 3, 4], dtype=torch.float32).reshape((1, 1, 4))
print('input:', input)
output = model(input)
print('out:', output)


input: tensor([[[1., 2., 3., 4.]]])
out: tensor([[[-1.2304, -0.1907, -0.1379,  1.5591]]],
       grad_fn=<NativeLayerNormBackward>)
